This code tests whether the implemented function is actually working properly or not

It does this by calculating the moments over a unit cube with vertices (0,0,0) and (1,1,1) and edges aligned with axes and comparing with the analytical exact solution of the moments which is:

1/((i+1)*(j+1)*(k+1))

In [1]:
import pymeshlab
import numpy as np

In [2]:
m = pymeshlab.Mesh(
    vertex_matrix = [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1 ,0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1 ,0],
        [1, 1 ,1]
    ],
    face_matrix = [
        [0,2,4],
        [4,2,6],
        [0,1,3],
        [0,3,2],
        [0,5,1],
        [0,4,5],
        [1,7,3],
        [1,5,7],
        [4,7,5],
        [4,6,7],
        [7,6,2],
        [7,2,3]
    ])

In [3]:
v_matrix = m.vertex_matrix()

In [4]:
f_matrix = m.face_matrix()

In [5]:
np.array(f_matrix)+1

array([[1, 3, 5],
       [5, 3, 7],
       [1, 2, 4],
       [1, 4, 3],
       [1, 6, 2],
       [1, 5, 6],
       [2, 8, 4],
       [2, 6, 8],
       [5, 8, 6],
       [5, 7, 8],
       [8, 7, 3],
       [8, 3, 4]], dtype=int32)

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [6]:
# Define the maximum order of moments to be calculated
max_m = 10

In [7]:
def C_ijk(C,i,j,k):
    return (C[0]**i)*(C[1]**j)*(C[2]**k)*np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))
        

In [8]:
def D_ijk(D_tensor,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # D_ijk=0
        pass
    elif (0==i) and (0==j) and (0==k):
        # D_ijk = 1
        D_tensor[i,j,k] = 1
    else:
        D_tensor[i,j,k] = B[0]*D_tensor[i-1,j,k]+B[1]*D_tensor[i,j-1,k]+B[2]*D_tensor[i,j,k-1]+C_ijk(C,i,j,k)
    

In [9]:
def S_ijk(S_tensor,D_tensor,A,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # S_ijk = 0
        pass
    elif (0==i) and (0==j) and (0==k):
        # S_ijk = 1
        S_tensor[i,j,k] = 1
    else:
        S_tensor[i,j,k] = A[0]*S_tensor[i-1,j,k]+A[1]*S_tensor[i,j-1,k]+A[2]*S_tensor[i,j,k-1]+D_tensor[i,j,k]
    

In [10]:
def vol(A,B,C):
    return 1./6*np.linalg.det([A,B,C])


In [11]:
def M_ijk(f_matrix, v_matrix, max_m):
    M_tensor = np.zeros([max_m,max_m,max_m])
    
    
    for face_num in range(len(f_matrix)):
        
        
        # Get the three coordinates A,B,C
        # Any mesh library should return these in CCW order
        [A,B,C] = v_matrix[f_matrix[face_num]]
        
        # Calculate the face moments tensor
        # Each face moments tensor requires calculating S_ijk, D_ijk, and C_ijk Tensor Calculations.

        # C_ijk tensor is not reused, calculate on the fly
        
        # D_ijk tensor is reused, calculate and store
        D_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        D_ijk(D_tensor,B,C,i,j,k)
                        
        # S_ijk tensor is reqused, calculate and store
        S_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        S_ijk(S_tensor,D_tensor,A,B,C,i,j,k)
                        
        # Now calculate the moments on current face
        f_M_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        f_M_tensor[i,j,k] = (np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3)*np.linalg.det([A,B,C])*S_tensor[i,j,k]
        
        # Add contribution to the total moments
        M_tensor = M_tensor+f_M_tensor
                
    return M_tensor



In [12]:
def M_ijk_fast(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    A = np.zeros([num_faces,3])
    B = np.zeros([num_faces,3])
    C = np.zeros([num_faces,3])
    for face_num in range(num_faces):
        [A[face_num,:],B[face_num,:],C[face_num,:]] = v_matrix[f_matrix[face_num]]
    # Calculate Determinants
    dets = np.linalg.det([[A[i],B[i],C[i]] for i in range(len(f_matrix))])
    
    # Allocate Tensors
    M_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    C_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    D_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    S_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (C[:,0]**i)*(C[:,1]**j)*(C[:,2]**k)*(np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = B[:,0]*D_tensor[:,i-1,j,k]+B[:,1]*D_tensor[:,i,j-1,k]+B[:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = A[:,0]*S_tensor[:,i-1,j,k]+A[:,1]*S_tensor[:,i,j-1,k]+A[:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3))*dets[:]*S_tensor[:,i,j,k]
    return np.sum(M_tensor,axis=0)
#      M_tensor = M_tensor
#     return M_tensor

In [21]:
moments = M_ijk_fast(f_matrix,v_matrix,max_m)

In [14]:
test_tensor = np.zeros(np.shape(moments))

In [15]:
for i in range(max_m):
    for j in range(max_m):
        for k in range(max_m):
            if (i+j+k)<=max_m:
                test_tensor[i,j,k] = 1/((i+1)*(j+1)*(k+1))

In [16]:
np.linalg.norm(test_tensor-moments)

1.8762299503193352e-16

Since the value is ~ machine precision, the code is working properly!

In [17]:
print(moments)

[[[1.         0.5        0.33333333 0.25       0.2        0.16666667
   0.14285714 0.125      0.11111111 0.1       ]
  [0.5        0.25       0.16666667 0.125      0.1        0.08333333
   0.07142857 0.0625     0.05555556 0.05      ]
  [0.33333333 0.16666667 0.11111111 0.08333333 0.06666667 0.05555556
   0.04761905 0.04166667 0.03703704 0.        ]
  [0.25       0.125      0.08333333 0.0625     0.05       0.04166667
   0.03571429 0.03125    0.         0.        ]
  [0.2        0.1        0.06666667 0.05       0.04       0.03333333
   0.02857143 0.         0.         0.        ]
  [0.16666667 0.08333333 0.05555556 0.04166667 0.03333333 0.02777778
   0.         0.         0.         0.        ]
  [0.14285714 0.07142857 0.04761905 0.03571429 0.02857143 0.
   0.         0.         0.         0.        ]
  [0.125      0.0625     0.04166667 0.03125    0.         0.
   0.         0.         0.         0.        ]
  [0.11111111 0.05555556 0.03703704 0.         0.         0.
   0.         0.   

In [18]:
v_matrix = np.array(v_matrix)

# Calculate the centered and scaled moments
v_matrix[:,0] = ( v_matrix[:,0] - moments[1,0,0]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
v_matrix[:,1] = ( v_matrix[:,1] - moments[0,1,0]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
v_matrix[:,2] = ( v_matrix[:,2] - moments[0,0,1]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
moments = M_ijk(f_matrix,v_matrix,max_m)
print(moments)

# Note how running this cell twice gives nicer results!

[[[ 1.00000000e+00 -1.14491749e-16  8.33333333e-02 -2.49366500e-17
    1.25000000e-02 -6.85757874e-18  2.23214286e-03 -1.52465931e-18
    4.34027778e-04 -4.13352078e-19]
  [-1.14491749e-16  1.73472348e-18 -8.67361738e-18  3.25260652e-19
   -1.30104261e-18  4.06575815e-20 -2.54109884e-19  5.08219768e-21
   -4.74338450e-20  2.11758237e-22]
  [ 8.33333333e-02 -8.89045781e-18  6.94444444e-03 -2.22261445e-18
    1.04166667e-03 -5.55653613e-19  1.86011905e-04 -1.43995601e-19
    3.61689815e-05  0.00000000e+00]
  [-2.60208521e-17  2.16840434e-19 -2.11419424e-18  5.42101086e-20
   -3.32036915e-19  8.47032947e-21 -5.92923063e-20  3.81164826e-21
    0.00000000e+00  0.00000000e+00]
  [ 1.25000000e-02 -1.40946282e-18  1.04166667e-03 -3.52365706e-19
    1.56250000e-04 -8.38562618e-20  2.79017857e-05  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [-6.93889390e-18  5.42101086e-20 -5.69206141e-19  1.01643954e-20
   -8.47032947e-20  4.23516474e-22  0.00000000e+00  0.00000000e+00
    0.00000000e+